In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

LOAD ALL MODEL RESULTS

In [ ]:
try:
    mlp_results = pd.read_csv('mlp_results.csv')
    autoencoder_results = pd.read_csv('autoencoder_results.csv')
    lstm_results = pd.read_csv('lstm_results.csv')
    cnn1d_results = pd.read_csv('cnn1d_results.csv')
    
    # Combine all results
    all_results = pd.concat([mlp_results, autoencoder_results, 
                             lstm_results, cnn1d_results], ignore_index=True)
    
    print("Successfully loaded all model results!")
    print("\n" + "="*70)
    print("COMPREHENSIVE RESULTS TABLE")
    print("="*70)
    print(all_results.to_string(index=False))
    
except FileNotFoundError as e:
    print(f"Error: Could not find results file - {e}")
    print("Please ensure all models have been trained and results saved.")
    exit()

Error: Could not find results file - [Errno 2] No such file or directory: 'mlp_results.csv'
Please ensure all models have been trained and results saved.


: 

PERFORMANCE COMPARISON VISUALIZATIONS

In [ ]:
# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# Metrics to compare
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
models = all_results['Model'].values

BAR CHART COMPARISON

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, metric in enumerate(metrics):
    values = all_results[metric].values
    colors = ['#1f77b4', '#2ca02c', '#9467bd', '#ff7f0e']
    
    axes[idx].bar(models, values, color=colors, alpha=0.8, edgecolor='black')
    axes[idx].set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel(metric, fontsize=12)
    axes[idx].set_ylim([0, 1.05])
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(values):
        axes[idx].text(i, v + 0.02, f'{v:.4f}', ha='center', fontsize=10)
    
    # Rotate x-axis labels
    axes[idx].tick_params(axis='x', rotation=45)

# Remove extra subplot
axes[-1].axis('off')

plt.tight_layout()
plt.savefig('model_comparison_bars.png', dpi=300, bbox_inches='tight')
plt.show()

RADAR CHART COMPARISON

In [ ]:
from math import pi

# Number of metrics
categories = metrics
N = len(categories)

# Compute angle for each axis
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Initialize plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# Plot each model
colors = ['#1f77b4', '#2ca02c', '#9467bd', '#ff7f0e']
for idx, model in enumerate(models):
    values = all_results.iloc[idx][metrics].values.tolist()
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=model, color=colors[idx])
    ax.fill(angles, values, alpha=0.15, color=colors[idx])

# Fix axis to go in the right order and start at 12 o'clock
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)

# Draw axis lines for each angle and label
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)

# Set y-axis limits
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=9)

# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
plt.title('Model Performance Radar Chart', size=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('model_comparison_radar.png', dpi=300, bbox_inches='tight')
plt.show()

HEATMAP COMPARISON

In [ ]:
# Prepare data for heatmap
heatmap_data = all_results[metrics].T
heatmap_data.columns = models

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, fmt='.4f', cmap='YlGnBu', 
            cbar_kws={'label': 'Score'}, linewidths=0.5, linecolor='gray',
            vmin=0, vmax=1)
plt.title('Model Performance Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Models', fontsize=12)
plt.ylabel('Metrics', fontsize=12)
plt.tight_layout()
plt.savefig('model_comparison_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

GROUPED BAR CHART

In [ ]:
x = np.arange(len(models))
width = 0.15

fig, ax = plt.subplots(figsize=(14, 8))

for i, metric in enumerate(metrics):
    values = all_results[metric].values
    ax.bar(x + i * width, values, width, label=metric, alpha=0.8)

ax.set_xlabel('Models', fontsize=12, fontweight='bold')
ax.set_ylabel('Scores', fontsize=12, fontweight='bold')
ax.set_title('Grouped Performance Comparison', fontsize=16, fontweight='bold')
ax.set_xticks(x + width * 2)
ax.set_xticklabels(models)
ax.legend(loc='lower right', fontsize=10)
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig('model_comparison_grouped.png', dpi=300, bbox_inches='tight')
plt.show()

STATISTICAL SUMMARY

In [ ]:
# Find best model for each metric
print("\nBest Model for Each Metric:")
print("-" * 70)
for metric in metrics:
    best_idx = all_results[metric].idxmax()
    best_model = all_results.loc[best_idx, 'Model']
    best_score = all_results.loc[best_idx, metric]
    print(f"{metric:15s}: {best_model:20s} ({best_score:.4f})")

# Overall best model (based on average performance)
print("\n" + "="*70)
print("OVERALL BEST MODEL (Average Performance)")
print("="*70)

all_results['Average'] = all_results[metrics].mean(axis=1)
best_overall_idx = all_results['Average'].idxmax()
best_overall_model = all_results.loc[best_overall_idx, 'Model']
best_overall_score = all_results.loc[best_overall_idx, 'Average']

print(f"\nBest Model: {best_overall_model}")
print(f"Average Score: {best_overall_score:.4f}")
print("\nDetailed Scores:")
for metric in metrics:
    score = all_results.loc[best_overall_idx, metric]
    print(f"  {metric:12s}: {score:.4f}")

RANKING TABLE

In [ ]:
ranking_df = pd.DataFrame()
ranking_df['Model'] = models

for metric in metrics:
    ranks = all_results[metric].rank(ascending=False).astype(int)
    ranking_df[metric] = ranks

ranking_df['Average Rank'] = ranking_df[metrics].mean(axis=1)
ranking_df = ranking_df.sort_values('Average Rank')

print("\n" + ranking_df.to_string(index=False))

PERCENTAGE DIFFERENCES

In [ ]:
difference_df = pd.DataFrame()
difference_df['Model'] = models

for metric in metrics:
    max_val = all_results[metric].max()
    differences = ((all_results[metric] - max_val) / max_val * 100).round(2)
    difference_df[metric] = differences.astype(str) + '%'

print("\n" + difference_df.to_string(index=False))